## Woking on Flight Price Prediction dataset.

link for dataset: https://machinehack.com/hackathons/predict_the_flight_ticket_price_hackathon/overview

In [276]:
# importing the necessary libraries.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [277]:
# Reading the files:

Combine_df = pd.read_excel('Data_Train.xlsx')

In [278]:
# Looking the dataset:

Combine_df.head(5)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [279]:
# Let's look at the size of big dataset:

Combine_df.shape

(10683, 11)

## Applying Feature Enginnering

In [280]:
# Understanding the Data:

Combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [281]:
# Checking for the missing values:
Combine_df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [282]:
# Missing columns data.
Combine_df[Combine_df['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [283]:
# Checking for the Other data having same source and destination.
Combine_df[(Combine_df['Source'] == 'Delhi') & (Combine_df['Destination'] == 'Cochin')]['Total_Stops'].value_counts()

1 stop      3185
2 stops     1113
non-stop     213
3 stops       25
Name: Total_Stops, dtype: int64

In [284]:
# Replacing missing value with the frequent value.

Combine_df['Total_Stops'] = Combine_df['Total_Stops'].fillna('1 stop')

In [285]:
Combine_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


#### 1. Breaking the full date column, into 3 columns

In [286]:
Combine_df['Date'] = Combine_df['Date_of_Journey'].str.split('/').str[0].astype(int)
Combine_df['Month'] = Combine_df['Date_of_Journey'].str.split('/').str[1].astype(int)
Combine_df['Year'] = Combine_df['Date_of_Journey'].str.split('/').str[2].astype(int)

# Droping Date_of_Journey column:
Combine_df.drop('Date_of_Journey', axis=1, inplace=True)

#### 2. Cleaning Arrival_Time column

In [287]:
# Replacing with the cleaned data.
Combine_df['Arrival_Time'] = Combine_df['Arrival_Time'].str.split(' ').str[0]

#### 3. Total_Stops :-  Replacing the text into numercal representation for Ordinal Encoding.

In [288]:
Combine_df['Total_Stops'] = Combine_df['Total_Stops'].replace('non-stop', '0')
Combine_df['Total_Stops'] = Combine_df['Total_Stops'].str.split(' ').str[0].astype(int)

#### 3. Duration :- Representing the data into minutes.

In [289]:
# Separating the hour and minute columns for appending into the orignal duration column.
Combine_df[['temp_hr','temp_min']] = Combine_df['Duration'].str.extract(r'(?:(\d+)h)?\s?(?:(\d+)m)?').fillna(0).astype(int)

In [290]:
Combine_df['Duration_in_min'] = Combine_df['temp_hr']*60 + Combine_df['temp_min']

In [291]:
# Deleting the temporary hours and minutes column along with original duration column.

Combine_df.drop(['Duration','temp_hr','temp_min'], axis=1, inplace=True)

#### 4. Additional_info

In [292]:
Combine_df['Additional_Info'].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [293]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Combine_df['Additional_Info'] = encoder.fit_transform(Combine_df['Additional_Info'])
Combine_df['Airline'] = encoder.fit_transform(Combine_df['Airline'])
Combine_df['Source'] = encoder.fit_transform(Combine_df['Source'])
Combine_df['Destination'] = encoder.fit_transform(Combine_df['Destination'])

In [294]:
Combine_df.corr()['Price']*100

Airline             -3.952047
Source               1.599783
Destination         -7.111212
Total_Stops         60.388306
Additional_Info     -6.547771
Price              100.000000
Date               -15.373825
Month              -10.365054
Year                      NaN
Duration_in_min     50.637148
Name: Price, dtype: float64

In [295]:
Combine_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Date,Month,Year,Duration_in_min
0,3,0,5,BLR → DEL,22:20,01:10,0,8,3897,24,3,2019,170
1,1,3,0,CCU → IXR → BBI → BLR,05:50,13:15,2,8,7662,1,5,2019,445
2,4,2,1,DEL → LKO → BOM → COK,09:25,04:25,2,8,13882,9,6,2019,1140
3,3,3,0,CCU → NAG → BLR,18:05,23:30,1,8,6218,12,5,2019,325
4,3,0,5,BLR → NAG → DEL,16:50,21:35,1,8,13302,1,3,2019,285


#### Dropping the Columns: (Route, Dep_Time, Arrival_Time)

In [296]:
Combine_df.drop(['Route','Dep_Time','Arrival_Time'], axis=1, inplace=True)

## Feature Selection

In [297]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [298]:
from sklearn.model_selection import train_test_split

X = Combine_df.drop(['Price'], axis=1)
y = Combine_df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [299]:
model = SelectFromModel(Lasso(alpha=0.05))

In [300]:
model.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.05))

In [301]:
model.get_support()

array([ True,  True,  True,  True,  True,  True,  True, False,  True])

In [302]:
X_train.columns[(model.get_support())]

Index(['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info',
       'Date', 'Month', 'Duration_in_min'],
      dtype='object')

In [303]:
X_train.drop(['Year'], axis=1, inplace=True)
X_test.drop(['Year'],axis=1, inplace=True)